In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import time

from numpy import array
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import completeness_score, homogeneity_score, v_measure_score
import imblearn
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.utils import resample

Using TensorFlow backend.


# <font color=black>Bootstrap</font>

In [2]:
df = pd.read_csv('dataset_lof.csv')
dataset = df.copy()
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,0.1
0,0.000809,0.001476,0.000009,0.000007,0.000005,1.556176e-07,0.001410,0.015054,0.005891,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
1,0.000809,0.000198,0.000005,0.000003,0.000004,1.266300e-07,0.002055,0.021935,0.008585,0.000000,...,5.960467e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
2,0.001221,0.708609,0.000032,0.000021,0.000029,1.769005e-05,0.014746,0.000000,0.008945,0.019363,...,3.725292e-08,0.000009,0.0,0.000009,0.000009,0.7075,0.0,0.7075,0.7075,1.0
3,0.000809,0.000002,0.000009,0.000007,0.000005,3.264917e-07,0.001209,0.012903,0.005050,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
4,0.000809,0.001209,0.000005,0.000003,0.000003,1.540919e-07,0.001692,0.018065,0.007070,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0


In [3]:
col_names = ["Destination_Port",
             "Flow_Duration", 
             "Total_Fwd_Packets",
             "Total_Backward_Packets",
             "Total_Length_of_Fwd_Packets", 
             "Total_Length_of_Bwd_Packets", 
             "Fwd_Packet_Length_Max", 
             "Fwd_Packet_Length_Min", 
             "Fwd_Packet_Length_Mean", 
             "Fwd_Packet_Length_Std",
             "Bwd_Packet_Length_Max", 
             "Bwd_Packet_Length_Min", 
             "Bwd_Packet_Length_Mean", 
             "Bwd_Packet_Length_Std",
             "Flow_Bytes_s", 
             "Flow_Packets_s", 
             "Flow_IAT_Mean", 
             "Flow_IAT_Std", 
             "Flow_IAT_Max", 
             "Flow_IAT_Min",
             "Fwd_IAT_Total", 
             "Fwd_IAT_Mean", 
             "Fwd_IAT_Std", 
             "Fwd_IAT_Max", 
             "Fwd_IAT_Min",
             "Bwd_IAT_Total", 
             "Bwd_IAT_Mean", 
             "Bwd_IAT_Std", 
             "Bwd_IAT_Max", 
             "Bwd_IAT_Min",
             "Fwd_PSH_Flags", 
             "Bwd_PSH_Flags", 
             "Fwd_URG_Flags", 
             "Bwd_URG_Flags", 
             "Fwd_Header_Length", 
             "Bwd_Header_Length",
             "Fwd_Packets_s", 
             "Bwd_Packets_s", 
             "Min_Packet_Length", 
             "Max_Packet_Length", 
             "Packet_Length_Mean", 
             "Packet_Length_Std", 
             "Packet_Length_Variance",
             "FIN_Flag_Count", 
             "SYN_Flag_Count", 
             "RST_Flag_Count", 
             "PSH_Flag_Count", 
             "ACK_Flag_Count", 
             "URG_Flag_Count", 
             "CWE_Flag_Count", 
             "ECE_Flag_Count", 
             "Down_Up_Ratio", 
             "Average_Packet_Size", 
             "Avg_Fwd_Segment_Size", 
             "Avg_Bwd_Segment_Size",
             "Fwd_Avg_Bytes_Bulk", 
             "Fwd_Avg_Packets_Bulk", 
             "Fwd_Avg_Bulk_Rate", 
             "Bwd_Avg_Bytes_Bulk", 
             "Bwd_Avg_Packets_Bulk",
             "Bwd_Avg_Bulk_Rate",
             "Subflow_Fwd_Packets", 
             "Subflow_Fwd_Bytes", 
             "Subflow_Bwd_Packets", 
             "Subflow_Bwd_Bytes",
             "Init_Win_bytes_forward", 
             "Init_Win_bytes_backward", 
             "act_data_pkt_fwd", 
             "min_seg_size_forward",
             "Active_Mean", 
             "Active_Std", 
             "Active_Max", 
             "Active_Min",
             "Idle_Mean", 
             "Idle_Std", 
             "Idle_Max", 
             "Idle_Min",
             "Label_Category_Num"
            ]

In [4]:
dataset.columns = col_names

In [5]:
dataset.head()

,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,...,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label_Category_Num
0,0.000809,0.001476,0.000009,0.000007,0.000005,1.556176e-07,0.001410,0.015054,0.005891,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
1,0.000809,0.000198,0.000005,0.000003,0.000004,1.266300e-07,0.002055,0.021935,0.008585,0.000000,...,5.960467e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
2,0.001221,0.708609,0.000032,0.000021,0.000029,1.769005e-05,0.014746,0.000000,0.008945,0.019363,...,3.725292e-08,0.000009,0.0,0.000009,0.000009,0.7075,0.0,0.7075,0.7075,1.0
3,0.000809,0.000002,0.000009,0.000007,0.000005,3.264917e-07,0.001209,0.012903,0.005050,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
4,0.000809,0.001209,0.000005,0.000003,0.000003,1.540919e-07,0.001692,0.018065,0.007070,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0


In [6]:
dataset["Label_Category_Num"] = dataset.Label_Category_Num.astype(int)

In [7]:
dataset.head()

,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,...,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label_Category_Num
0,0.000809,0.001476,0.000009,0.000007,0.000005,1.556176e-07,0.001410,0.015054,0.005891,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0
1,0.000809,0.000198,0.000005,0.000003,0.000004,1.266300e-07,0.002055,0.021935,0.008585,0.000000,...,5.960467e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0
2,0.001221,0.708609,0.000032,0.000021,0.000029,1.769005e-05,0.014746,0.000000,0.008945,0.019363,...,3.725292e-08,0.000009,0.0,0.000009,0.000009,0.7075,0.0,0.7075,0.7075,1
3,0.000809,0.000002,0.000009,0.000007,0.000005,3.264917e-07,0.001209,0.012903,0.005050,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0
4,0.000809,0.001209,0.000005,0.000003,0.000003,1.540919e-07,0.001692,0.018065,0.007070,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0


In [8]:
benign_df      = dataset[dataset['Label_Category_Num'] == 0]
dos_df         = dataset[dataset['Label_Category_Num'] == 1]
probe_df       = dataset[dataset['Label_Category_Num'] == 2]
ddos_df        = dataset[dataset['Label_Category_Num'] == 3] 
brute_force_df = dataset[dataset['Label_Category_Num'] == 4]
botnet_df      = dataset[dataset['Label_Category_Num'] == 5]
web_attack_df  = dataset[dataset['Label_Category_Num'] == 6]

In [9]:
bittlein = pd.concat([dos_df, probe_df,ddos_df, brute_force_df, web_attack_df, botnet_df])
dos_input = bittlein.iloc[:,:-1]
output = bittlein.iloc[:,[-1]]
dos_input.tail()

,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,...,act_data_pkt_fwd,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
2543603,0.123293,8.578975e-03,0.000014,0.000010,0.000000e+00,2.746192e-08,0.000000,0.000000,0.00000,0.0,...,0.0,5.215409e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2543765,0.123293,8.440933e-03,0.000014,0.000010,0.000000e+00,2.746192e-08,0.000000,0.000000,0.00000,0.0,...,0.0,5.215409e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2543790,0.123293,8.538583e-03,0.000014,0.000010,0.000000e+00,2.746192e-08,0.000000,0.000000,0.00000,0.0,...,0.0,5.215409e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2543921,0.048920,5.666667e-07,0.000005,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.00101,0.0,...,0.0,3.725292e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2544063,0.030411,3.333333e-08,0.000005,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.00101,0.0,...,0.0,3.725292e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
output.tail()

,Label_Category_Num
2543603,5
2543765,5
2543790,5
2543921,5
2544063,5


In [11]:
test = pd.concat([dos_input, output], axis=1)

In [12]:
n_iterations = 5
n_size = int(len(bittlein))*(3)

medians = list()
for i in range(n_iterations):
    r = resample(test, n_samples = n_size)
r.tail()
r['Label_Category_Num'].value_counts()

1    705765
2    456431
3    373655
4     39107
6      6178
5      5408
Name: Label_Category_Num, dtype: int64

In [13]:
final = pd.concat([test, r],axis=0)

In [14]:
bootstrap = pd.concat([benign_df, final])

In [15]:
bootstrap['Label_Category_Num'].value_counts()

0    2016179
1     941009
2     608602
3     498201
4      52141
6       8227
5       7212
Name: Label_Category_Num, dtype: int64

In [ ]:
bootstrap.to_csv("dataset_bootstrap.csv", index = False)